In [1]:
import pandas as pd 
import numpy as np 

# Open the csv files

In [2]:
orders=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\orders.csv")
order_full_detail=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\order_products__prior.csv")
train=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\order_products__train.csv")
departments=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\departments.csv")
aisles=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\aisles.csv")
products=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\products.csv")

# Look inside of the files

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
order_full_detail.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [6]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [7]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [9]:
products.shape

(49688, 4)

In [10]:
train.shape[0]+order_full_detail.shape[0]

33819106

# Merging the training and the prior dataset together for full dataset

In [11]:
order_full_detail=pd.concat([train,order_full_detail],ignore_index=True)

In [12]:
order_full_detail.shape

(33819106, 4)

In [13]:
products=products.merge(aisles,on="aisle_id",how="left")
products=products.merge(departments,on="department_id",how="left")
products.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


# We will combine features from other dataset to this one

In [14]:
dataframe=products.copy()

# Adding new features to the dataframe

remove the unneccessary order column

In [15]:
order_full_detail.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [16]:
order_full_detail.product_id.unique().sum()

1234416216

In [17]:
products.product_id.sum()

1234473516

meaning there are some of the products and their details missing
- Products are listed but has not been sold

In [18]:
order_full_detail.drop(columns=["add_to_cart_order"],inplace=True)

we want to find out how many times the product has been reordered while purchasing

In [19]:
order_full_detail.groupby(by="product_id")["reordered"].sum().reset_index()

,product_id,reordered
0,1,1185
1,2,13
2,3,209
3,4,161
4,5,10
...,...,...
49680,49684,1
49681,49685,6
49682,49686,89
49683,49687,6


Lets see a sample

In [20]:
order_full_detail[order_full_detail["product_id"]==49685]["reordered"].sum()

6

We want to find out how many times the product has been purchased

In [21]:
order_full_detail.groupby(by="product_id")["order_id"].nunique().reset_index()

,product_id,order_id
0,1,1928
1,2,94
2,3,283
3,4,351
4,5,16
...,...,...
49680,49684,9
49681,49685,49
49682,49686,127
49683,49687,14


We can see  sample

In [22]:
order_full_detail[order_full_detail["product_id"]==49684] # 9 times this product has been purchased

,order_id,product_id,reordered
6705549,561618,49684,0
9977484,907116,49684,0
14344982,1367981,49684,1
23142864,2295026,49684,0
25580779,2551882,49684,0
25880810,2583499,49684,0
28244457,2832300,49684,0
29200897,2933365,49684,0
32059556,3235517,49684,0


In [23]:
dataframe.shape

(49688, 6)

In [24]:
col1=order_full_detail.groupby(by="product_id")["reordered"].sum().reset_index()
col2=order_full_detail.groupby(by="product_id")["order_id"].nunique().reset_index()
print(col1.shape)
print(col2.shape)

(49685, 2)
(49685, 2)


## Finding and Filling Missing rows

Left Join (how="left"): A left join ensures that all rows from the left DataFrame (products) are retained. If there's a matching row in the right DataFrame (reordered_sum), the corresponding values will be added. If there's no match, the result will have NaN values for columns from the right DataFrame.

In [25]:
dataframe=dataframe.merge(col1,on="product_id",how="left")
dataframe=dataframe.merge(col2,on="product_id",how="left")

In [26]:
dataframe.head()

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,1185.0,1928.0
1,2,All-Seasons Salt,104,13,spices seasonings,pantry,13.0,94.0
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,209.0,283.0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen,161.0,351.0
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry,10.0,16.0


In [27]:
dataframe[dataframe.isna().any(axis=1)]

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id
3629,3630,Protein Granola Apple Crisp,57,14,granola,breakfast,NaN,NaN
7044,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13,spreads,pantry,NaN,NaN
46624,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7,refrigerated,beverages,NaN,NaN


In [28]:
dataframe.fillna(0,inplace=True)

In [29]:
dataframe[dataframe.isna().any(axis=1)]

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id


## Droping unimportant rows and columns

In [30]:
old_columns=dataframe.columns.tolist()
old_columns

['product_id',
 'product_name',
 'aisle_id',
 'department_id',
 'aisle',
 'department',
 'reordered',
 'order_id']

In [31]:
new_columns=['product_id','product_name','aisle_id','department_id','aisle','department','reorder_total','order_total']

In [32]:
dataframe.columns=new_columns
dataframe.drop(columns=["product_name"],inplace=True)
dataframe.set_index("product_id",inplace=True)
dataframe.head(10)

,aisle_id,department_id,aisle,department,reorder_total,order_total
product_id,,,,,,
1,61,19,cookies cakes,snacks,1185.0,1928.0
2,104,13,spices seasonings,pantry,13.0,94.0
3,94,7,tea,beverages,209.0,283.0
4,38,1,frozen meals,frozen,161.0,351.0
5,5,13,marinades meat preparation,pantry,10.0,16.0
6,11,11,cold flu allergy,personal care,3.0,8.0
7,98,7,juice nectars,beverages,13.0,31.0
8,116,1,frozen produce,frozen,90.0,178.0
9,120,16,yogurt,dairy eggs,84.0,161.0


## Encode and Standardiz and Increasing Columns for Better Clusturing (Neighbour algorithm)

In [33]:
dataframe.dtypes

aisle_id           int64
department_id      int64
aisle             object
department        object
reorder_total    float64
order_total      float64
dtype: object

In [34]:
numaric_columns=dataframe.select_dtypes(include=["int64","float64"]).columns.tolist()
categorical_columns=dataframe.select_dtypes(include=["object"]).columns.tolist()
categorical_columns

['aisle', 'department']

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
onehot=OneHotEncoder(handle_unknown="ignore")

numaric_pipeline=Pipeline(
    [
        ("scaler",scaler)
    ]
)
catagorical_pipeline=Pipeline(
    [
        ("onehot",onehot)
    ]
)

transformer=ColumnTransformer(
    [
        ("numaric",numaric_pipeline,numaric_columns),
        ("onehot",catagorical_pipeline,categorical_columns),
    ]
)


In [36]:
Matrix=transformer.fit_transform(dataframe)

Matrix

<49688x159 sparse matrix of type '<class 'numpy.float64'>'
	with 298128 stored elements in Compressed Sparse Row format>

# ML Algorithm Train

In [37]:
from sklearn.neighbors import NearestNeighbors

n_nearest=NearestNeighbors(n_neighbors=5)
n_nearest.fit(Matrix)

NearestNeighbors()

In [44]:
dataframe.head()

,aisle_id,department_id,aisle,department,reorder_total,order_total
product_id,,,,,,
1,61,19,cookies cakes,snacks,1185.0,1928.0
2,104,13,spices seasonings,pantry,13.0,94.0
3,94,7,tea,beverages,209.0,283.0
4,38,1,frozen meals,frozen,161.0,351.0
5,5,13,marinades meat preparation,pantry,10.0,16.0


In [45]:
distance, suggestions=n_nearest.kneighbors(Matrix[0],n_neighbors=5)  # distance, row index = model.kneighbours()
suggestions

array([[    0, 36306, 18149,  6316, 23191]], dtype=int64)

In [46]:
for i in suggestions.ravel():
    print(f'''{products.at[i,"product_name"]}  ------- {products.at[i,"aisle"]}  --------- {products.at[i,"department"]}''')

Chocolate Sandwich Cookies  ------- cookies cakes  --------- snacks
Milano Double Chocolate Cookies  ------- cookies cakes  --------- snacks
Oatmeal Raisin Cookies  ------- cookies cakes  --------- snacks
Vanilla Sandwich Creme Cookies  ------- cookies cakes  --------- snacks
Chocolate Whole Grain Graham Snacks Bunny Grahams  ------- cookies cakes  --------- snacks


In [47]:
dataframe.shape

(49688, 6)